**GRADIENT BOOSTING**

 It takes an iterative approach by combining weak learners to create strong learner by focusing on mistakes of prior iterations

Uses the boosting method
Pros:
    - Extremely Powerful
    - Accepts various types of inputs
    - Can be used for classification
    - Outputs feature importance
Cons:
    - Longer to train(Can't parallelize')
    - More difficult to properly tune
    - More likely to overfit

In [15]:
import string 
import pandas as pd
import nltk
from nltk.tokenize import wordpunct_tokenize,word_tokenize
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

dataset = pd.read_csv(os.getcwd()+'\\Ex_Files_NLP_Python_ML_EssT\\Exercise Files\\Ch01\\01_03\\Start\\SMSSpamCollection.tsv', header=None, sep='\t')
dataset.columns = ['label', 'body_text']

#create new features containing the percentage of punctuation in text and the body length of text 

def count_punctuation_and_body_len(text):
    punct_count = [char for char in text if char in string.punctuation]
    body_len = len(text) - text.count(" ")
    punct_perc = round(len(punct_count)/body_len, 3)*100
    return body_len, punct_perc

dataset['body_len'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[0])
dataset['punc%'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[1])

sp_words = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
def clean_data(text):
    tokens = ''.join([char for char in text if char not in string.punctuation])
    tokens_punct = re.split('\s+', tokens)
    tokens_stop = [token.lower()  for token in tokens_punct if token not in sp_words]
    clean_text = [ps.stem(word) for word in tokens_stop]
    return clean_text

dataset['cleaned_text'] = dataset['body_text'].apply(lambda x: clean_data(x))
    
tfidf_vect = TfidfVectorizer(analyzer=clean_data)
x_tfidf = tfidf_vect.fit_transform(dataset['body_text'])
dataset_frame = pd.DataFrame(x_tfidf.toarray())
x_tfidf_features = pd.concat([dataset[['body_len','punc%']], dataset_frame], axis=1)
x_count_features = CountVectorizer.(analyzer=clean_data),fit_trasnform(dataset['body_text'])
x_tfidf_features.head()

SyntaxError: invalid syntax (<ipython-input-15-f64afcc14cd4>, line 38)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score

In [7]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_decision_function', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_init_decision_function', '_init_state', '_is_initialized', '_make_estimator', '_resize_state', '_staged_decision_function', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'n_features', 'predict', 'predict_log_proba', 'predict_proba', 'score', 's

In [13]:
X_train, x_test, y_train, y_test = train_test_split(x_tfidf_features, dataset['label'], test_size=0.2)
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred = gb_model.predict(x_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print ('n_est: {} / max_dept: {} / learning_rate: {} ----- precision: {} / recall: {} / accuracy: {}'.format(
                                    est, max_depth, lr, round(precision, 3), round(recall, 3),
                                    round((y_test == y_pred).sum()/len(y_pred), 3)))

In [ ]:
for n_est in [50, 100,150]:
    for max_depth in [3,7,11,15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)

C:\Users\user\Anaconda3\Lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


n_est: 50 / max_dept: 3 / learning_rate: 0.01 ----- precision: 0.0 / recall: 0.0 / accuracy: 0.873
n_est: 50 / max_dept: 3 / learning_rate: 0.1 ----- precision: 0.946 / recall: 0.739 / accuracy: 0.961
n_est: 50 / max_dept: 3 / learning_rate: 1 ----- precision: 0.884 / recall: 0.754 / accuracy: 0.956
n_est: 50 / max_dept: 7 / learning_rate: 0.01 ----- precision: 1.0 / recall: 0.007 / accuracy: 0.873
n_est: 50 / max_dept: 7 / learning_rate: 0.1 ----- precision: 0.883 / recall: 0.796 / accuracy: 0.961
n_est: 50 / max_dept: 7 / learning_rate: 1 ----- precision: 0.836 / recall: 0.824 / accuracy: 0.957
n_est: 50 / max_dept: 11 / learning_rate: 0.01 ----- precision: 0.0 / recall: 0.0 / accuracy: 0.873
n_est: 50 / max_dept: 11 / learning_rate: 0.1 ----- precision: 0.892 / recall: 0.817 / accuracy: 0.964
n_est: 50 / max_dept: 11 / learning_rate: 1 ----- precision: 0.85 / recall: 0.838 / accuracy: 0.961
n_est: 50 / max_dept: 15 / learning_rate: 0.01 ----- precision: 0.0 / recall: 0.0 / accuracy:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection imort GridSearchCV

In [ ]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators':[100,150]
    'max_depth':[7,11,15]
    'learning_rate':[0.1]
}
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(x_tfidf_features, dataset['lable'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [17]:
gb = GradientBoostingClassifier()
param = {
    'n_estimators':[100,150],
    'max_depth':[7,11,15],
    'learning_rate':[0.1]
}
gs = GridSearchCV(gb, param, cv=5, n_jobs=-1)
cv_fit = gs.fit(x_count_features, dataset['lable'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

NameError: name 'GridSearchCV' is not defined